In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Paridade Put - Call
#### (europeia)
    1. rf : anual
    2. venc_du : dias uteis
    3. S0 : ativo objeto spot
    4. strike: preço de exercicio opcao



In [9]:
premio_c = 0.5
premio_p = 1.2

strike = 23 
venc = 10 # Vencimento da estratégia em anos
rf = 0.1375 # *100 '% a.a'
m_rf = 0
S0 = 10


#### Call

In [10]:
## Descobrir "c" (Prêmio da Call)
c_0 = premio_p + S0 - (strike)/((1+rf)**venc)
# c_0 = premio_p + S0 - (strike)*np.exp(-rf * t)

#### Put

In [11]:
## Descobrir "p" (Prêmio da Put)
p_0 = premio_c + (strike)/((1+rf)**venc) - S0
# p_0 = premio_c + (strike)*np.exp(-rf * t) - S0

## Modelo Binário

In [12]:
def mod_binario_q(cenario_up, cenario_down, S0, strike, rf, tipo, ret = False):
    
    u = cenario_up/S0
    d = cenario_down/S0

    q = (1 + rf - d)/(u - d)

    if tipo == "call":
        payoff_up = cenario_up - strike if cenario_up > strike else 0
        payoff_down = cenario_down - strike if cenario_down > strike else 0
    else:
        payoff_up = strike - cenario_up if cenario_up > strike else 0
        payoff_down = strike - cenario_down if cenario_down > strike else 0

    premio = (q * payoff_up + (1-q)* payoff_down)/(1+rf)

    if ret == "payoff":
        return premio
    else:
        return premio


# # # Raiz "Modelo Binário por 'q' "
# INPUTS
cenario_up = 60
cenario_down = 40
S0 = 50
strike = 50
rf = 0.1
tipo = "call"
# #

u = cenario_up/S0
d = cenario_down/S0

q = (1 + rf - d)/(u - d)

if tipo == "call":
    payoff_up = cenario_up - strike if cenario_up > strike else 0
    payoff_down = cenario_down - strike if cenario_down > strike else 0
else:
    payoff_up = strike - cenario_up if cenario_up > strike else 0
    payoff_down = strike - cenario_down if cenario_down > strike else 0

premio = (q * payoff_up + (1-q)* payoff_down)/ (1+rf)

In [23]:
def mod_binario_pr(cenario_up, cenario_down, S0, strike, rf, tipo, ret = False):
    
    u = cenario_up/S0
    d = cenario_down/S0

    if tipo == "call":
        payoff_up = cenario_up - strike if cenario_up > strike else 0
        payoff_down = cenario_down - strike if cenario_down > strike else 0
    else:
        payoff_up = strike - cenario_up if cenario_up > strike else 0
        payoff_down = strike - cenario_down if cenario_down > strike else 0

    delta = (payoff_up - payoff_down) / ((u-d)* S0)
    B = (u * payoff_down - d * payoff_up) / (u  -d) * (1 + rf)

    V0 = B + delta * S0

    if ret == "payoff":
        return [payoff_up, payoff_down]
    else:
        return V0
    

# # # Raiz "Modelo Binário por 'pr' "
# INPUTS
cenario_up = 140
cenario_down = 60
S0 = 100
strike = 59
rf = 0.1
tipo = "call"
# #

u = cenario_up/S0
d = cenario_down/S0

if tipo == "call":
    payoff_up = cenario_up - strike if cenario_up > strike else 0
    payoff_down = cenario_down - strike if cenario_down > strike else 0
else:
    payoff_up = strike - cenario_up if strike > cenario_up else 0
    payoff_down = strike - cenario_down if strike > cenario_down else 0

delta = (payoff_up - payoff_down) / ((u - d)* S0)
B = (u * payoff_down - d * payoff_up) / ((u  - d) * (1 + rf))

V0 = B + delta * S0

print(delta)

1.0


In [14]:
def port_repl(u, d, rf, S0, payoff_up, payoff_down, t_rf = 1):
    delta = (payoff_up - payoff_down) / ((u - d)* S0)
    B = (u * payoff_down - d * payoff_up) / ((u  - d) * ((1 + rf)**t_rf))
    V0 = B + delta * S0
    return V0

In [15]:
print(delta)

0.5000000000000001


## Multiplos Períodos

In [16]:
def mod_binario_mult_deltaB(cenarios, S0, strike, rf, tipo):
    sz = len(cenarios)-1
    u = (np.max(cenarios)/S0)**(1/(len(cenarios)+1))
    d = (np.min(cenarios)/S0)**(1/(len(cenarios)+1))

    while (sz+1)*sz/2 > 1:
        sz = len(cenarios)-1
        Vn = [ ]
        for i in range(sz):
            if tipo =="call":
                payoff_up = cenarios[i] - strike if cenarios[i] > strike else 0
                payoff_down = cenarios[i+1] - strike if cenarios[i+1] > strike else 0
            else:
                payoff_up = strike - cenarios[i] if cenarios[i] < strike else 0
                payoff_down = strike - cenarios[i+1] if cenarios[i+1] < strike else 0
            Vn += port_repl(u, d, rf, S0, payoff_up, payoff_down)
            print(Vn)
        cenarios = Vn
        if len(cenarios) > 1:
            sz = len(cenarios) - 1

    return cenarios

In [17]:
port_repl(1.2,0.8,0.1,20,10,0)

6.81818181818182

In [18]:
mod_binario_mult_deltaB([144, 96, 64], 100,100, 0.1, "put")

[]
[]


[]

## 2 Períodos

In [1]:
# # # Raiz "Modelo Binário por 'pr' "
# INPUTS
cenario_up = 196
cenario_mid = 84
cenario_down = 36
S0 = 100
strike = 84
rf = 0.1
tipo = "put"
# #

u = (cenario_up/S0)**(1/2)
d = (cenario_down/S0)**(1/2)

if tipo == "call":
    payoff_up = cenario_up - strike if cenario_up > strike else 0
    payoff_midu = cenario_mid - strike if cenario_mid > strike else 0
    payoff_down = cenario_down - strike if cenario_down > strike else 0
else:
    payoff_up = strike - cenario_up if cenario_up < strike else 0
    payoff_mid = strike - cenario_mid if cenario_mid < strike else 0
    payoff_down = strike - cenario_down if cenario_down < strike else 0

delta_up = (payoff_up - payoff_mid) / ((u - d)* S0*u)
delta_down = (payoff_mid - payoff_down) / ((u - d)* S0*d)

B_up = (u * payoff_mid - d * payoff_up) / ((u  - d) * (1 + rf))
B_down = (u * payoff_down - d * payoff_mid) / ((u  - d) * (1 + rf))

V1u = B_up + delta_up * S0*u
V1d = B_down + delta_down * S0*d

delta = (V1u - V1d)/((u-d)*S0)
B = (u * V1d - d * V1u) / ((u  - d) * (1 + rf))

V0 = B + delta * S0

In [2]:
V0

5.5785123966942045

## 3 Períodos

In [33]:
# # # Raiz "Modelo Binário por 'pr' "
# INPUTS
cenario_up = 144
cenario_midu = 96
cenario_midd = 96
cenario_down = 64
S0 = 100
strike = 100
rf = 0.1
tipo = "put"
# #

u = (cenario_up/S0)**(1/3)
d = (cenario_down/S0)**(1/3)

if tipo == "call":
    payoff_up = cenario_up - strike if cenario_up > strike else 0
    payoff_midu = cenario_midu - strike if cenario_midu > strike else 0
    payoff_midd = cenario_midd - strike if cenario_midd > strike else 0
    payoff_down = cenario_down - strike if cenario_down > strike else 0
else:
    payoff_up = strike - cenario_up if cenario_up < strike else 0
    payoff_midu = strike - cenario_midu if cenario_midu < strike else 0
    payoff_midd = strike - cenario_midd if cenario_midd < strike else 0
    payoff_down = strike - cenario_down if cenario_down < strike else 0

delta_up = (payoff_up - payoff_midu) / ((u - d)* S0 * u * u)
delta_mid = (payoff_midu - payoff_midd) / ((u - d)* S0 * u * d)
delta_down = (payoff_midd - payoff_down) / ((u - d)* S0 * d * d)

B_up = (u * payoff_midu - d * payoff_up) / ((u  - d) * (1 + rf))
B_mid = (u * payoff_midd - d * payoff_midu) / ((u  - d) * (1 + rf))
B_down = (u * payoff_down - d * payoff_midd) / ((u  - d) * (1 + rf))

V2u = B_up + delta_up * S0 * u * u
V2m = B_mid + delta_mid * S0 * u * d
V2d = B_down + delta_down * S0 * d * d



delta_u = (V2u - V2m) / ((u - d)* S0 * u )
delta_d = (V2m - V2d) / ((u - d)* S0 * d )

B_u = (u * V2m - d * V2u) / ((u  - d) * (1 + rf))
B_d = (u * V2d - d * V2m) / ((u  - d) * (1 + rf))

V1u = B_u + delta_u * S0 * u
V1d = B_d + delta_d * S0 * d



delta = (V1u - V1d)/((u-d) * S0)
B = (u * V1d - d * V1u) / ((u  - d) * (1 + rf))

V0 = B + delta * S0

In [8]:
venc = 1
x = 202
s0 = 50
rf = 0.01
# vi > 0
vi = x-s0
c = vi / (1+rf)
c

150.4950495049505